In [3]:
from __future__ import absolute_import
from datetime import datetime, timedelta, date
import hashlib
import requests
import os
import sys
from os.path import *
import urllib
import xlrd
sys.path.insert(0, '../')
try:
    from bs4 import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration, Doctype)
    _DECLARATION_OR_DOCTYPE = (Declaration, Doctype)
except ImportError:
    from BeautifulSoup import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration)
    _DECLARATION_OR_DOCTYPE = Declaration
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from pyvirtualdisplay.display import Display
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time #using this library to pause or "sleep" program

In [4]:
from selenium.webdriver.support import expected_conditions as EC 

In [ ]:
#Steps (broad overview)
#1. login to gmail (use skytruth email)
#2. open correct email (Echo Notify - need to make an account)
#3. Click on link in email 
#4. Scrape detailed facility report (still working on this)

In [5]:
#Open gmail - set up driver, navigate to website 
driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')
driver.implicitly_wait(60) # web driver waits 60 seconds before throwing a NoSuchElementException - applies to every action - only have to write this once. 
driver.maximize_window()
#driver.get('https://mail.google.com')

driver.get('https://echo.epa.gov/detailed-facility-report?fid=CABAA00006013A0010')

C:\Users\Tatiana\AppData\Local\Temp\ipykernel_25124\2321693607.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')


In [5]:
#Login to gmail - enter username, press next, enter password

In [28]:
#enter username
driver.find_element(By.ID, "identifierId").send_keys("tatiana@skytruth.org") #enter your username 
#click the 'next' button
driver.find_element(By.ID, "identifierNext").click()

In [29]:
#enter password
driver.find_element(By.NAME, "Passwd").send_keys("#######") #enter your password
#click the 'next' button 
driver.find_element(By.ID, "passwordNext").click()

In [ ]:
#may need to manually enter a verification code and press "next" button to complete login.

In [ ]:
#need to add a "wait" here, because usually get sent an email about a security alert, and then the following code has errors executing.

In [30]:
#should also maybe make this into a function.
#Will eventually need to edit this to comb through  multiple pages if necesssary, if desired email is not on first page.
email_titles=driver.find_elements(By.XPATH, "//span[@class='bog']") #create list; all email subjects are of class 'bog'
for title in email_titles: #now comb through list; if email subject matches our desired email subject, click the email.
    if title.text == "EPA ECHO Notify: Air Results for SEP 24, 2022 - OCT 01, 2022": #if the subject in the list matches our desired email subject...
        subject=title.text
        title.click()
        print("Email clicked:", subject) 
    

Email clicked: EPA ECHO Notify: Air Results for SEP 24, 2022 - OCT 01, 2022


In [31]:
wait = WebDriverWait(driver, 10) #explicit wait - used to tell the Web Driver to wait for certain conditions (Expected Conditions) or maximum time exceeded before throwing “ElementNotVisibleException” exception

In [32]:
original_tab_email = driver.current_window_handle #store id of the original tab (the email tab)

In [34]:
#click on link
#Notes: The <a> tag defines a hyperlink, which is used to link from one page to another.
#The most important attribute of the <a> element is the href attribute, which indicates the link's destination.

#will need to find a way to automate this process rather than having to manually enter the link text - we'll figure that out later. 
#will also need to deal with a list of links. 
driver.find_element(By.LINK_TEXT,'CABAA00006013A0010').click()

# Wait for the new window or tab (selenium views windows and tabs as being the same)
wait.until(EC.number_of_windows_to_be(2))
# Loop through until we find a new window handle
for window_handle in driver.window_handles:
    if window_handle != original_tab_email:
        driver.switch_to.window(window_handle)
        break
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))

In [14]:
#facility info 

#can't put this one in the loop because the xpath is different.
facility_address=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div/div").text
print(facility_address)

facility_info_row_labels=["FRS ID: ","EPA Region: ", "Latitude: ", "Longitude: ", "Locational Data Source: ", "Industries: ", "Indian country: "]

for i in range(1, 8, 1): #(initialization, stop (noninclusive), increment)
    facility_info_row=driver.find_element(By.XPATH, "//section[@class='echo-subsection echo-grid-two-thirds']/div/dl/div["+str(i)+"]/dd").text
    facility_info_row_label=facility_info_row_labels[i-1]
    print(facility_info_row_label, facility_info_row)

CHEVRON REFINERY RICHMOND
841 CHEVRON WAY, RICHMOND, CA 94801
FRS ID:  110020506460
EPA Region:  09
Latitude:  37.938779
Longitude:  -122.396453
Locational Data Source:  EIS
Industries:  Merchant Wholesalers, Nondurable Goods
Indian country:  N


In [ ]:
#scrape first two rows (row heading and violation info underneath) for the high priority violation. 

In [55]:
#it appears that there are always 11 columns, but just in case, i'll count them up before i start scraping info. 
columns=driver.find_elements(By.XPATH, "//table[@id='enforcementAndComplianceSummary']/thead/tr/th")
max_columns=len(columns)+1 #add 1 - since there are 11 columns, we need 12 for our noninclusive stop in the loop.
print(max_columns)

12


In [56]:
for i in range(1, max_columns, 1): #(initialization, stop (noninclusive), increment)
    header_label=driver.find_element(By.XPATH, "//table[@id='enforcementAndComplianceSummary']/thead/tr/th["+str(i)+"]").text #finds text inside the column header cell 
    cell=driver.find_element(By.XPATH, "//table[@id='enforcementAndComplianceSummary']/tbody/tr/td["+str(i)+"]").text #finds text inside the row cell 
    print(header_label, ": ", cell, sep = '') #sep='' avoids unnecessary whitespaces between header_label and colon.

Statute: CAA
Compliance Monitoring Activities (5 years): 14
Date of Last Compliance Monitoring Activity: 08/02/2022
Compliance Status: High Priority Violation
Qtrs with NC (of 12): 12
Qtrs with Significant Violation: 12
Informal Enforcement Actions (5 years): 340
Formal Enforcement Actions (5 years): 169
Penalties from Formal Enforcement Actions (5 years): $685,500
EPA Cases (5 years): 1
Penalties from EPA Cases (5 years): $1,930,300
